In [1]:
import os
import pandas as pd
import json
import numpy as np
#!pip install python-dotenv
from dotenv import load_dotenv
load_dotenv()

True

In [ ]:
#from ipynb.fs.full.my_functions import factorial

In [2]:
import tweepy
import traceback

In [3]:
import spacy
import torch

#!pip install spacymoji
from spacymoji import Emoji

from sklearn.svm import SVC
from sklearn.utils import resample
from sklearn.impute import SimpleImputer
from sklearn.dummy import DummyClassifier
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.model_selection import GroupShuffleSplit, train_test_split
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler, FunctionTransformer

#!pip install transformers
from transformers import AutoModel, BertTokenizerFast

from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score
from sklearn.metrics import fbeta_score

In [5]:
def evaluation_summary(description, true_labels, predictions):
  print("Evaluation for: " + description)
  print(classification_report(true_labels, predictions,  digits=3, zero_division=0))
  print('\nConfusion matrix:\n',confusion_matrix(true_labels, predictions))

# Import, Fix and Clean Data

In [6]:
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
from nltk.corpus import stopwords
import re
from nltk.stem.porter import PorterStemmer
from collections import Counter

ps = PorterStemmer()
wnl = nltk.stem.WordNetLemmatizer()

stop_words = stopwords.words('english')
stopwords_dict = Counter(stop_words)

#remove_col = ['threadid','tweetid','support', 'evidentiality', 'certainty', 'event']
remove_col = ['Unnamed: 0.1', 'Unnamed: 0', 'id', 'author_id', 'edit_history_tweet_ids', 'article_id', 'Unnamed: 0.2']

# Impute null values with None
def null_process(feature_df):
    col = 'text'
    feature_df.loc[feature_df[col].isnull(), col] = "None"
    return feature_df

# Removed unused clumns
def remove_unused_col(df,column_n=remove_col):
    df = df.drop(column_n,axis=1)
    return df

def clean_dataset(df):
    # remove unused column
    df = remove_unused_col(df)
    #impute null values
    df = null_process(df)
    return df

# Cleaning text from unused characters
def clean_text(text):
    text = str(text).replace(r'http[\w:/\.]+', ' ')  # removing urls
    text = str(text).replace(r'[^\.\w\s]', ' ')  # remove everything but characters and punctuation
    text = str(text).replace('[^a-zA-Z]', ' ')
    text = str(text).replace(r'\s\s+', ' ')
    text = text.lower().strip()
    #text = ' '.join(text)    
    return text

## Nltk Preprocessing includes:
# Stop words, Stemming and Lemmetization
def nltk_preprocess(text):
    text = clean_text(text)
    wordlist = re.sub(r'[^\w\s]', '', text).split()
    text = ' '.join([word for word in wordlist if word not in stopwords_dict])
    text = [ps.stem(word) for word in wordlist if not word in stopwords_dict]
    text = ' '.join([wnl.lemmatize(word) for word in wordlist if word not in stopwords_dict])
    return text

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\karli\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\karli\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [127]:
X.head()

,text,source,created_at,lang,description,username,verified,name,location
163133,私は 花火とほろよい で夏を楽しみたいシーンを選んでツイートすると沢尻さんが詠む夏の一句が届...,Twitter for Android,2018-07-23T03:45:49.000Z,ja,NaN,kirakirah0ppy,False,⭐きらきら⭐,NaN
204478,responsibility require one hundred percent pre...,Twittascope,2018-06-08T23:00:38.000Z,en,She got an ass that could swallow up a g-string,amy1thomas,False,lembit opik,NaN
203226,ironic reality starsarah palins cpac speech tr...,Twitter for Websites,2013-03-17T18:39:43.000Z,en,"M.D., pediatrics/adolescents. Aspiring comedia...",burnedoutdoc,False,J.J. Shaughnessy,"Los Angeles, CA, USA"
124057,ease living downtown austin paired luxury amen...,Facebook,2018-07-01T23:43:14.000Z,en,"austin tx realtor, wife, mom, wine lover, coff...",SusanAvant,False,Susan Avant,"Austin, TX"
202652,r v issue rt attackerman printout roll call am...,TweetDeck,2013-07-24T23:47:22.000Z,en,I write/edit @techdirt. I survived for 12 year...,mmasnick,True,Mike Masnick,NaN


## Load Model

In [18]:
import textwrap

# Wrap text to 80 characters.
wrapper = textwrap.TextWrapper(width=80) 

print(wrapper.fill(sen_w_feats[1]))

I created this Tweet at 2018-02-05T20:30:01.000Z o'clock in the en language and
in  from Twitter Web Client. My name is 17thReese, username is 17th_reese and I
am not verified. I would describe myself as . I wrote: NFL Lawyer, Who Claimed
Super Bowl Is ‘Rigged’, Found Dead #Topbuzz https://t.co/a9Lbddvxya


In [21]:
from transformers import BertTokenizer

# Load the BERT tokenizer.
print('Loading BERT tokenizer...')
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)


Loading BERT tokenizer...


In [16]:
from transformers import BertForSequenceClassification
from transformers import BertConfig

config = BertConfig.from_pretrained(
    'bert-base-uncased', 
    num_labels=2 # The number of output labels--2 for binary classification
)
config.gradient_checkpointing = True

# Load BertForSequenceClassification, the pretrained BERT model with a single 
# linear classification layer on top. 
model = BertForSequenceClassification.from_pretrained(
    "bert-base-uncased", # Use the 12-layer BERT model, with an uncased vocab
    config = config,
)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [11]:
max_len = 480

# # For every sentence...
# for sent in sen_w_feats:

#     # Tokenize the text and add `[CLS]` and `[SEP]` tokens.
#     input_ids = tokenizer.encode(sent, add_special_tokens=True)

#     # Update the maximum sentence length.
#     max_len = max(max_len, len(input_ids))

print('Max sentence length: ', max_len) # 474 from updating

Max sentence length:  480


In [19]:
# Larger batch sizes tend to be better, and we can fit this in memory.
batch_size = 8

# I used a smaller learning rate to combat over-fitting that I was seeing in the
# validation loss. I could probably try even smaller.
learning_rate = 1e-5

# Number of training epochs. 
epochs = 4

max_len = 480

In [22]:
# Tokenize all of the sentences and map the tokens to thier word IDs.
input_ids = []
attention_masks = []

print('Encoding all reviews in the dataset...')

# For every sentence...
for sent in sen_w_feats:
    # `encode_plus` will:
    #   (1) Tokenize the sentence.
    #   (2) Prepend the `[CLS]` token to the start.
    #   (3) Append the `[SEP]` token to the end.
    #   (4) Map tokens to their IDs.
    #   (5) Pad or truncate the sentence to `max_length`
    #   (6) Create attention masks for [PAD] tokens.
    encoded_dict = tokenizer.encode_plus(
                        sent,                      # Sentence to encode.
                        add_special_tokens = True, # Add '[CLS]' and '[SEP]'
                        max_length = max_len,           # Pad & truncate all sentences.
                        truncation = True,
                        padding = 'max_length',
                        return_attention_mask = True,   # Construct attn. masks.
                        return_tensors = 'pt',     # Return pytorch tensors.
                )
    
    # Add the encoded sentence to the list.    
    input_ids.append(encoded_dict['input_ids'])
    
    # And its attention mask (simply differentiates padding from non-padding).
    attention_masks.append(encoded_dict['attention_mask'])

# Convert the lists into tensors.
input_ids = torch.cat(input_ids, dim=0)
attention_masks = torch.cat(attention_masks, dim=0)
labels = torch.tensor(labels)

print('DONE.')

Encoding all reviews in the dataset...
DONE.


C:\Users\karli\AppData\Local\Temp\ipykernel_32592\2228480261.py:35: DeprecationWarning: In future, it will be an error for 'np.bool_' scalars to be interpreted as an index
  labels = torch.tensor(labels)


In [23]:
from torch.utils.data import TensorDataset

# Split the samples, and create TensorDatasets for each split. 
train_dataset = TensorDataset(input_ids[train_ix], attention_masks[train_ix], labels[train_ix])
val_dataset = TensorDataset(input_ids[val_ix], attention_masks[val_ix], labels[val_ix])
test_dataset = TensorDataset(input_ids[test_ix], attention_masks[test_ix], labels[test_ix])

print(len(train_dataset), len(val_dataset), len(test_dataset), len(train_dataset) + len(val_dataset) + len(test_dataset))

150164 9324 58374 217862


In [24]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

# Create the DataLoaders for our training and validation sets.
# We'll take training samples in random order. 
train_dataloader = DataLoader(
            train_dataset,  # The training samples.
            sampler = RandomSampler(train_dataset), # Select batches randomly
            batch_size = batch_size # Trains with this batch size.
        )

# For validation the order doesn't matter, so we'll just read them sequentially.
validation_dataloader = DataLoader(
            val_dataset, # The validation samples.
            sampler = SequentialSampler(val_dataset), # Pull out batches sequentially.
            batch_size = batch_size # Evaluate with this batch size.
        )

In [25]:
from torch.optim import AdamW

# Note: AdamW is a class from the huggingface library (as opposed to pytorch) 
# I believe the 'W' stands for 'Weight Decay fix"
optimizer = AdamW(model.parameters(),
                  lr = learning_rate, 
                  eps = 1e-8 
                )

In [26]:
from transformers import get_linear_schedule_with_warmup

# Total number of training steps is [number of batches] x [number of epochs]. 
# (Note that this is not the same as the number of training samples!)
total_steps = len(train_dataloader) * epochs

# Create the learning rate scheduler.
scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = 0, # Default value in run_glue.py
                                            num_training_steps = total_steps)

In [27]:
import time
import datetime

def format_time(elapsed):
    '''
    Takes a time in seconds and returns a string hh:mm:ss
    '''
    # Round to the nearest second.
    elapsed_rounded = int(round((elapsed)))
    
    # Format as hh:mm:ss
    return str(datetime.timedelta(seconds=elapsed_rounded))

In [1]:
import torch

# If there's a GPU available...
if torch.cuda.is_available():    

    # Tell PyTorch to use the GPU.    
    device = torch.device("cuda")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())

    print('We will use the GPU:', torch.cuda.get_device_name(0))

# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

No GPU available, using the CPU instead.


## Training Feature-Engineered BERT

In [29]:
import numpy as np

# Function to calculate the accuracy of our predictions vs labels
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

In [32]:
# print(b_input_ids[:3])
# print(b_input_ids.shape)
# print(b_labels.shape)
encoded_labels = torch.nn.functional.one_hot(labels.long()).float() #.astype(np.float32)
#print(encoded_labels[:20])

# Split the samples, and create TensorDatasets for each split. 
train_dataset = TensorDataset(input_ids[train_ix], attention_masks[train_ix], encoded_labels[train_ix])
val_dataset = TensorDataset(input_ids[val_ix], attention_masks[val_ix], encoded_labels[val_ix])
test_dataset = TensorDataset(input_ids[test_ix], attention_masks[test_ix], encoded_labels[test_ix])

train_dataloader = DataLoader(
            train_dataset,  # The training samples.
            sampler = RandomSampler(train_dataset), # Select batches randomly
            batch_size = batch_size # Trains with this batch size.
        )

# For validation the order doesn't matter, so we'll just read them sequentially.
validation_dataloader = DataLoader(
            val_dataset, # The validation samples.
            sampler = SequentialSampler(val_dataset), # Pull out batches sequentially.
            batch_size = batch_size # Evaluate with this batch size.
        )

In [33]:
import random
import numpy as np

# This training code is based on the `run_glue.py` script here:
# https://github.com/huggingface/transformers/blob/5bfcd0485ece086ebcbed2d008813037968a9e58/examples/run_glue.py#L128

# Set the seed value all over the place to make this reproducible.
seed_val = 111019

random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

# We'll store a number of quantities such as training and validation loss, 
# validation accuracy, and timings.
training_stats = []

# Measure the total training time for the whole run.
total_t0 = time.time()

# For each epoch...
for epoch_i in range(0, epochs):
    
    # ========================================
    #               Training
    # ========================================
    
    # Perform one full pass over the training set.

    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')

    # Measure how long the training epoch takes.
    t0 = time.time()

    # Reset the total loss for this epoch.
    total_train_loss = 0

    # Put the model into training mode. Don't be mislead--the call to 
    # `train` just changes the *mode*, it doesn't *perform* the training.
    # `dropout` and `batchnorm` layers behave differently during training
    # vs. test (source: https://stackoverflow.com/questions/51433378/what-does-model-train-do-in-pytorch)
    model.train()

    # For each batch of training data...
    for step, batch in enumerate(train_dataloader):

        # Progress update every 40 batches.
        if step % 40 == 0 and not step == 0:
            # Calculate elapsed time in minutes.
            elapsed = format_time(time.time() - t0)
            
            # Report progress.
            print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_dataloader), elapsed))

        # Unpack this training batch from our dataloader. 
        #
        # As we unpack the batch, we'll also copy each tensor to the GPU using the 
        # `to` method.
        #
        # `batch` contains three pytorch tensors:
        #   [0]: input ids 
        #   [1]: attention masks
        #   [2]: labels 
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)

        # Always clear any previously calculated gradients before performing a
        # backward pass. PyTorch doesn't do this automatically because 
        # accumulating the gradients is "convenient while training RNNs". 
        # (source: https://stackoverflow.com/questions/48001598/why-do-we-need-to-call-zero-grad-in-pytorch)
        model.zero_grad()        

        # Perform a forward pass (evaluate the model on this training batch).
        # In PyTorch, calling `model` will in turn call the model's `forward` 
        # function and pass down the arguments. The `forward` function is 
        # documented here: 
        # https://huggingface.co/transformers/model_doc/bert.html#bertforsequenceclassification
        # The results are returned in a results object, documented here:
        # https://huggingface.co/transformers/main_classes/output.html#transformers.modeling_outputs.SequenceClassifierOutput
        # Specifically, we'll get the loss (because we provided labels) and the
        # "logits"--the model outputs prior to activation.
        #print(b_input_ids)
        #print(b_labels)
        result = model(b_input_ids, 
                       token_type_ids=None, 
                       attention_mask=b_input_mask, 
                       labels=b_labels,
                       return_dict=True)

        loss = result.loss
        logits = result.logits

        # Accumulate the training loss over all of the batches so that we can
        # calculate the average loss at the end. `loss` is a Tensor containing a
        # single value; the `.item()` function just returns the Python value 
        # from the tensor.
        total_train_loss += loss.item()

        # Perform a backward pass to calculate the gradients.
        loss.backward()

        # Clip the norm of the gradients to 1.0.
        # This is to help prevent the "exploding gradients" problem.
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        
        # Update the learning rate.
        scheduler.step()

        # Update parameters and take a step using the computed gradient.
        # The optimizer dictates the "update rule"--how the parameters are
        # modified based on their gradients, the learning rate, etc.
        optimizer.step()


    # Calculate the average loss over all of the batches.
    avg_train_loss = total_train_loss / len(train_dataloader)            
    
    # Measure how long this epoch took.
    training_time = format_time(time.time() - t0)

    print("")
    print("  Average training loss: {0:.2f}".format(avg_train_loss))
    print("  Training epcoh took: {:}".format(training_time))
        
    # ========================================
    #               Validation
    # ========================================
    # After the completion of each training epoch, measure our performance on
    # our validation set.

    print("")
    print("Running Validation...")

    t0 = time.time()

    # Put the model in evaluation mode--the dropout layers behave differently
    # during evaluation.
    model.eval()

    # Tracking variables 
    total_eval_accuracy = 0
    total_eval_loss = 0
    nb_eval_steps = 0

    # Evaluate data for one epoch
    for batch in validation_dataloader:
        
        # Unpack this training batch from our dataloader. 
        #
        # As we unpack the batch, we'll also copy each tensor to the GPU using 
        # the `to` method.
        #
        # `batch` contains three pytorch tensors:
        #   [0]: input ids 
        #   [1]: attention masks
        #   [2]: labels 
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)
        
        # Tell pytorch not to bother with constructing the compute graph during
        # the forward pass, since this is only needed for backprop (training).
        with torch.no_grad():        

            # Forward pass, calculate logit predictions.
            # token_type_ids is the same as the "segment ids", which 
            # differentiates sentence 1 and 2 in 2-sentence tasks.
            result = model(b_input_ids, 
                           token_type_ids=None, 
                           attention_mask=b_input_mask,
                           labels=b_labels,
                           return_dict=True)

        # Get the loss and "logits" output by the model. The "logits" are the 
        # output values prior to applying an activation function like the 
        # softmax.
        loss = result.loss
        logits = result.logits
            
        # Accumulate the validation loss.
        total_eval_loss += loss.item()

        # Move logits and labels to CPU
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()

        # Calculate the accuracy for this batch of test sentences, and
        # accumulate it over all batches.
        total_eval_accuracy += flat_accuracy(logits, label_ids)
        

    # Report the final accuracy for this validation run.
    avg_val_accuracy = total_eval_accuracy / len(validation_dataloader)
    print("  Accuracy: {0:.2f}".format(avg_val_accuracy))

    # Calculate the average loss over all of the batches.
    avg_val_loss = total_eval_loss / len(validation_dataloader)
    
    # Measure how long the validation run took.
    validation_time = format_time(time.time() - t0)
    
    print("  Validation Loss: {0:.2f}".format(avg_val_loss))
    print("  Validation took: {:}".format(validation_time))

    # Record all statistics from this epoch.
    training_stats.append(
        {
            'epoch': epoch_i + 1,
            'Training Loss': avg_train_loss,
            'Valid. Loss': avg_val_loss,
            'Valid. Accur.': avg_val_accuracy,
            'Training Time': training_time,
            'Validation Time': validation_time
        }
    )

print("")
print("Training complete!")

print("Total training took {:} (h:mm:ss)".format(format_time(time.time()-total_t0)))


======== Epoch 1 / 4 ========
Training...


C:\Users\karli\AppData\Local\Programs\Python\Python39\lib\site-packages\torch\optim\lr_scheduler.py:138: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  warnings.warn("Detected call of `lr_scheduler.step()` before `optimizer.step()`. "


  Batch    40  of  18,771.    Elapsed: 0:25:12.


KeyboardInterrupt: 

In [7]:
# Combine dataset classes
real_df = pd.read_csv("data/FakeNewsNet/dataset/pf_real.csv")
fake_df = pd.read_csv("data/FakeNewsNet/dataset/pf_fake.csv")
fake_df['true'] = False
real_df['true'] = True
df = pd.concat([fake_df, real_df])
df.reset_index()

#print(df.head())


# Fix imbalance
# https://elitedatascience.com/imbalanced-classes
# Separate majority and minority classes
df_majority = df[df.true==True]
df_minority = df[df.true==False]
 
# Downsample majority class
df_majority_downsampled = resample(df_majority, 
                                 replace=False,    # sample without replacement
                                 n_samples=len(df_minority),     # to match minority class
                                 random_state=123) # reproducible results
 
# Combine minority class with downsampled majority class
df_downsampled = pd.concat([df_majority_downsampled, df_minority])

df_downsampled = df_downsampled.sample(frac=1, random_state=111019)

df_downsampled.reset_index(inplace=True)



# https://mccormickml.com/2021/06/29/combining-categorical-numerical-features-with-bert/#31-all-features-to-text
# This will hold all of the dataset samples, as strings.
sen_w_feats = []

# The labels for the samples.
labels = []

# First, reload the dataset to undo the transformations we applied for XGBoost.
data_df = df_downsampled.copy()

# Some of the reviews are missing either a "Title" or "Review Text", so we'll 
# replace the NaN values with empty string.
data_df = data_df.fillna("")

print('Combining features into strings...')

# text 	source 	created_at 	lang 
# description 	username 	verified 	name 	location

# For each of the samples...
#print(data_df.head())
print(len(data_df))
for index, row in data_df.iterrows():
    # Piece it together...    
    combined = ""
    
    combined += f"I created this Tweet at {row['created_at']} o'clock " + \
        f"in the {row['lang']} language and in {row['location']} " + \
        f"from {row['source']}. "
        
    verification_status = "verified" if row["verified"] else "not verified"
    combined += f"My name is {row['name']}, username is {row['username']} " + \
        f"and I am {verification_status}. "
    
    combined += f"I would describe myself as {row['description']}. "
    
    combined += f"I wrote: {row['text']} "
    
    # Add the combined text to the list.
    sen_w_feats.append(combined)

    # Also record the sample's label.
    #print(index)
    labels.append(data_df.iloc[index]['true'])

print('  DONE.')

print('Dataset contains {:,} samples.'.format(len(sen_w_feats)))



# Split train-test
# https://stackoverflow.com/questions/61337373/split-on-train-and-test-separating-by-group

# Update df to the downsampled one

X = df_downsampled.drop(labels='true', axis=1)
y = df_downsampled.true

#print(y.head())

# NLTK preprocessing - not necessary with BERT
#X['text'] = X.text.apply(nltk_preprocess)

X['created_at'] = pd.to_datetime(X['created_at']) # str to datetime
X['created_at'] = X.apply(lambda row: row['created_at'].hour, axis=1) # get hour from datetime


# Split into train/val/test sets
# Technique gained from https://stackoverflow.com/questions/68152473/python-split-to-train-test-val-using-groupshufflesplit
# Start by splitting into train-val and test

gs_1 = GroupShuffleSplit(n_splits=1, train_size=.7, random_state=111019)
train_val_ix, test_ix = next(gs_1.split(X, y, groups=X.article_id))

X_test = X.iloc[test_ix]
y_test = y.iloc[test_ix]

X_train_val = X.iloc[train_val_ix]
y_train_val = y.iloc[train_val_ix]

# Then split train-val into train and val (but reset index first)
X_train_val = X_train_val.reset_index(drop=True)
y_train_val = y_train_val.reset_index(drop=True)
gs_2 = GroupShuffleSplit(n_splits=1, train_size=0.9, random_state=111019)
train_ix, val_ix = next(gs_2.split(X_train_val, y_train_val, groups=X_train_val.article_id))

# Clean dataset, e.g., remove article_id column
X_train_val = clean_dataset(X_train_val)
X_test = clean_dataset(X_test)

X_train = X_train_val.iloc[train_ix]
y_train = y.iloc[train_ix]
X_val = X_train_val.iloc[val_ix]
y_val = y.iloc[val_ix]

Combining features into strings...
217862
  DONE.
Dataset contains 217,862 samples.


In [50]:
# Save model
import pickle
pickle.dump(lr_model_tfidf, open("simple_model.sav", "wb"))